In [2]:
import matplotlib.pyplot as plt
import os
import librosa
import essentia.standard as es
import numpy as np
from pylab import plot, show, figure, imshow
import ffmpeg
import sys
import subprocess
import IPython.display as ipd
import matplotlib
import essentia
from collections import defaultdict
import json
import pandas as pd
import acoustid as ai
import jdc
import math 
;

2023-04-25 18:58:34.636372: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-04-25 18:58:35.083876: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 18:58:35.085500: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-04-25 18:58:35.143801: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-25 18:58:35.143846: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does 

''

Class defining any audio file from an episode

In [3]:
# Universal
class Audio:
    def __init__(self, folder, file, chopped=False, start=0, end=0):
        self.dir = folder
        self.filename, self.ext = os.path.splitext(file)
        self.file = os.path.join(folder, file)
        
        self.chopped = chopped
        self.start = start
        self.end = end
        self.__process_text()
        # print(f"Processing S{self.season}E{self.episode} - {self.title}")
    
    def __process_text(self):
        self.season = self.filename[15:17]
        self.episode = self.filename[18:20]
        self.title = self.filename[23:].split(" [")[0]

In [4]:
%%add_to Audio
def process_essentia(self):
    if self.chopped:
        self.es_loader = es.EasyLoader(filename=self.file, startTime=self.start, endTime=self.end)()
        self.duration = self.end - self.start
    else:
        self.es_loader = es.MonoLoader(filename=self.file)()
        self.duration = len(self.es_loader)/44100
    self.extract_data()
    self.__get_bpm()
    self.__get_other_descriptors()
    # self.get_preds()
    # self.get_five_s_preds()
    # self.__create_metronome_file()

def extract_data(self):
    extractor = es.MusicExtractor()
    self.features, self.features_frames = extractor(self.file)

def __get_bpm(self):
    rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
    self.avg_bpm, self.beats, self.beats_confidence, self.bpm_estimates, self.beats_intervals = rhythm_extractor(self.es_loader)
def __get_other_descriptors(self):
    danceability = es.Danceability()
    self.danceability = danceability(self.es_loader)
    dynamicComplexity = es.DynamicComplexity()
    self.dynamic_complexity = dynamicComplexity(self.es_loader)
    levelExtractor=es.LevelExtractor()
    self.levels = levelExtractor(self.es_loader)

def get_five_s_preds(self):
    self.pred_sr=16000
    if self.chopped:
        self.pred_loader = es.EasyLoader(filename=self.file, startTime=self.start, endTime=self.
        end, sampleRate=self.pred_sr)()
    else:
        self.pred_loader = es.MonoLoader(filename=self.file, sampleRate=self.pred_sr)()
    self.five_s_preds = es.TensorflowPredictMusiCNN(patchHopSize=155,patchSize=311,
        graphFilename='msd-musicnn-1.pb')(self.pred_loader)

def get_preds(self):
    self.pred_sr=16000
    if self.chopped:
        self.pred_loader = es.EasyLoader(filename=self.file, startTime=self.start, endTime=self.
        end, sampleRate=self.pred_sr)()
    else:
        self.pred_loader = es.MonoLoader(filename=self.file, sampleRate=self.pred_sr)()
    self.preds = es.TensorflowPredictMusiCNN(
        graphFilename='msd-musicnn-1.pb')(self.pred_loader)
def get_five_s_preds(self):
    self.pred_sr=16000
    if self.chopped:
        self.pred_loader = es.EasyLoader(filename=self.file, startTime=self.start, endTime=self.
        end, sampleRate=self.pred_sr)()
    else:
        self.pred_loader = es.MonoLoader(filename=self.file, sampleRate=self.pred_sr)()
    self.five_s_preds = es.TensorflowPredictMusiCNN(patchHopSize=155,patchSize=311,
        graphFilename='msd-musicnn-1.pb')(self.pred_loader)

#new
def get_five_s_tempo(self):
    self.pred_sr=11025
    if self.chopped:
        self.temp_loader = es.EasyLoader(filename=self.file, startTime=self.start, endTime=self.
        end, sampleRate=self.pred_sr)()
    else:
        self.temp_loader = es.MonoLoader(filename=self.file, sampleRate=self.pred_sr)()
    self.global_bpm, self.local_bpm, self.local_probs = es.TempoCNN(lastPatchMode="repeat", patchHopSize=107,
        graphFilename='deeptemp-k16-3.pb')(self.temp_loader)

In [5]:
AUDIO="audio"
VIDEO ="video"
audio_files = os.listdir(AUDIO) 
all_audio = [Audio(AUDIO, audio_file) for audio_file in audio_files]
relevant_titles = ['Dreams', 'Nightmares', 'Bliss', 'Love', 'Family', 'Conflict', 'Coping', 'Death', 'Paradise']
relevant_episodes = [audio for audio in all_audio if audio.title in relevant_titles]
;

''

In [ ]:
for episode in relevant_episodes:
    episode.process_essentia()
    episode.get_five_s_tempo() ;
;

In [ ]:
print(relevant_episodes[0].features.descriptorNames())

['lowlevel.average_loudness', 'lowlevel.barkbands_crest.dmean', 'lowlevel.barkbands_crest.dmean2', 'lowlevel.barkbands_crest.dvar', 'lowlevel.barkbands_crest.dvar2', 'lowlevel.barkbands_crest.max', 'lowlevel.barkbands_crest.mean', 'lowlevel.barkbands_crest.median', 'lowlevel.barkbands_crest.min', 'lowlevel.barkbands_crest.stdev', 'lowlevel.barkbands_crest.var', 'lowlevel.barkbands_flatness_db.dmean', 'lowlevel.barkbands_flatness_db.dmean2', 'lowlevel.barkbands_flatness_db.dvar', 'lowlevel.barkbands_flatness_db.dvar2', 'lowlevel.barkbands_flatness_db.max', 'lowlevel.barkbands_flatness_db.mean', 'lowlevel.barkbands_flatness_db.median', 'lowlevel.barkbands_flatness_db.min', 'lowlevel.barkbands_flatness_db.stdev', 'lowlevel.barkbands_flatness_db.var', 'lowlevel.barkbands_kurtosis.dmean', 'lowlevel.barkbands_kurtosis.dmean2', 'lowlevel.barkbands_kurtosis.dvar', 'lowlevel.barkbands_kurtosis.dvar2', 'lowlevel.barkbands_kurtosis.max', 'lowlevel.barkbands_kurtosis.mean', 'lowlevel.barkbands_kur

In [ ]:
print(relevant_episodes[0].features_frames.descriptorNames())

['lowlevel.average_loudness', 'lowlevel.dynamic_complexity', 'lowlevel.loudness_ebu128.integrated', 'lowlevel.loudness_ebu128.loudness_range', 'metadata.audio_properties.analysis.equal_loudness', 'metadata.audio_properties.analysis.length', 'metadata.audio_properties.analysis.sample_rate', 'metadata.audio_properties.analysis.start_time', 'metadata.audio_properties.bit_rate', 'metadata.audio_properties.length', 'metadata.audio_properties.lossless', 'metadata.audio_properties.number_channels', 'metadata.audio_properties.replay_gain', 'metadata.audio_properties.sample_rate', 'rhythm.bpm', 'rhythm.bpm_histogram_first_peak_bpm', 'rhythm.bpm_histogram_first_peak_weight', 'rhythm.bpm_histogram_second_peak_bpm', 'rhythm.bpm_histogram_second_peak_spread', 'rhythm.bpm_histogram_second_peak_weight', 'rhythm.danceability', 'rhythm.onset_rate', 'tonal.chords_changes_rate', 'tonal.chords_number_rate', 'tonal.key_edma.strength', 'tonal.key_krumhansl.strength', 'tonal.key_temperley.strength', 'tonal.t

In [ ]:
e = relevant_episodes[0]
f = e.features_frames
d = f.descriptorNames()
print(e.duration)
for trait in d:
    a = relevant_episodes[0].features_frames[trait]
    if isinstance(a, np.ndarray):
        print(trait, a.shape)
    elif isinstance(a, str):
        print(trait, a)
    else:
        print(trait, a)
print((relevant_episodes[0].features_frames['lowlevel.dissonance'].shape[0]-1) * 2048 * .5/44100)

685.8879818594104
lowlevel.average_loudness 0.4253152310848236
lowlevel.dynamic_complexity 3.4685263633728027
lowlevel.loudness_ebu128.integrated -24.4328556060791
lowlevel.loudness_ebu128.loudness_range 7.149408340454102
metadata.audio_properties.analysis.equal_loudness 0.0
metadata.audio_properties.analysis.length 685.8880004882812
metadata.audio_properties.analysis.sample_rate 44100.0
metadata.audio_properties.analysis.start_time 0.0
metadata.audio_properties.bit_rate 232663.0
metadata.audio_properties.length 685.8880004882812
metadata.audio_properties.lossless 0.0
metadata.audio_properties.number_channels 2.0
metadata.audio_properties.replay_gain 1.144540786743164
metadata.audio_properties.sample_rate 48000.0
rhythm.bpm 111.05408477783203
rhythm.bpm_histogram_first_peak_bpm 110.0
rhythm.bpm_histogram_first_peak_weight 0.3160173296928406
rhythm.bpm_histogram_second_peak_bpm 117.0
rhythm.bpm_histogram_second_peak_spread 0.5662251710891724
rhythm.bpm_histogram_second_peak_weight 0.102

AttributeError: 'str' object has no attribute 'length'

In [6]:
e = relevant_episodes[0]
f = e.features
d = f.descriptorNames()
print(e.duration)
for trait in d:
    a = relevant_episodes[0].features[trait]
    if ( 
        trait[-3:] == "var"
    or  trait[-4:] == "var2"
    or  trait[-3:] == "max"
    or  trait[-3:] == "min"
    or  trait[-5:] == "mean2"
    or  trait[-6:] == "median"
    or  trait[-5:] == "stdev"
    ):
        continue
    if isinstance(a, np.ndarray):
        print(trait, a.shape)
    # elif isinstance(a, str):
    #     print(trait, len(a.len)
    else:
        print(trait, a)


685.8879818594104
lowlevel.average_loudness 0.4253152310848236
lowlevel.barkbands_crest.dmean 2.768580675125122
lowlevel.barkbands_crest.mean 12.091906547546387
lowlevel.barkbands_flatness_db.dmean 0.029547668993473053
lowlevel.barkbands_flatness_db.mean 0.23404230177402496
lowlevel.barkbands_kurtosis.dmean 6.5921807289123535
lowlevel.barkbands_kurtosis.mean 9.582996368408203
lowlevel.barkbands_skewness.dmean 0.7236010432243347
lowlevel.barkbands_skewness.mean 1.7882130146026611
lowlevel.barkbands_spread.dmean 4.32412052154541
lowlevel.barkbands_spread.mean 12.384190559387207
lowlevel.dissonance.dmean 0.031293995678424835
lowlevel.dissonance.mean 0.43146371841430664
lowlevel.dynamic_complexity 3.4685263633728027
lowlevel.erbbands_crest.dmean 2.766965389251709
lowlevel.erbbands_crest.mean 10.865649223327637
lowlevel.erbbands_flatness_db.dmean 0.02128393016755581
lowlevel.erbbands_flatness_db.mean 0.23147383332252502
lowlevel.erbbands_kurtosis.dmean 1.8822002410888672
lowlevel.erbbands_k

In [ ]:
relevant_overall_features = [
    'lowlevel.average_loudness',
    'lowlevel.dynamic_complexity', 
    'lowlevel.dissonance.mean', 
    'lowlevel.dissonance.stdev', 
    'lowlevel.spectral_energyband_low.mean',
    'lowlevel.spectral_energyband_low.stdev',
    'lowlevel.spectral_energyband_middle_low.mean',
    'lowlevel.spectral_energyband_middle_low.stdev',
    'lowlevel.spectral_energyband_middle_high.mean',
    'lowlevel.spectral_energyband_middle_high.stdev',
    'lowlevel.spectral_energyband_high.mean',
    'lowlevel.spectral_energyband_high.stdev',
    # 'rhythm.bpm',
    'rhythm.onset_rate',
    'rhythm.beats_loudness.mean',
    'rhythm.beats_loudness.stdev',
    'tonal.key_edma.key',
    'tonal.key_edma.scale',
    'tonal.key_edma.strength',
    'tonal.chords_key',
    'tonal.chords_scale',
    'tonal.chords_strength.mean',
    'tonal.chords_strength.stdev',
    'tonal.chords_changes_rate',
    'lowlevel.pitch_salience.mean',
    'lowlevel.pitch_salience.stdev',
    'lowlevel.silence_rate_60dB.mean',
    'lowlevel.silence_rate_60dB.stdev',
    'lowlevel.loudness_ebu128.integrated',
    'lowlevel.zerocrossingrate.mean',
    'lowlevel.zerocrossingrate.stdev',
    'lowlevel.melbands_flatness_db.mean',
    'lowlevel.melbands_flatness_db.stdev',
    'lowlevel.hfc.mean',
    'lowlevel.hfc.stdev',
    'lowlevel.spectral_centroid.mean',
    'lowlevel.spectral_centroid.stdev',
    'lowlevel.spectral_spread.mean',
    'lowlevel.spectral_spread.stdev',
    'lowlevel.spectral_rms.mean',
    'lowlevel.spectral_rms.stdev',
    'lowlevel.spectral_energy.mean',
    'lowlevel.spectral_energy.stdev',
    'lowlevel.spectral_flux.mean',
    'lowlevel.spectral_flux.stdev',
    'lowlevel.spectral_complexity.mean',
    'lowlevel.spectral_complexity.stdev',
    'lowlevel.spectral_entropy.mean',
    'lowlevel.spectral_entropy.stdev',
    'lowlevel.spectral_kurtosis.mean',
    'lowlevel.spectral_kurtosis.stdev'
    # 'lowlevel.spectral_skewness.mean',
    # 'lowlevel.spectral_rolloff.mean',
    # 'lowlevel.spectral_strongpeak.mean',
    # 'lowlevel.spectral_decrease.mean',
] 

In [ ]:
relevant_features = ['lowlevel.dissonance', 
                     'lowlevel.melbands_crest',
                     'lowlevel.melbands_flatness_db',
                     'lowlevel.pitch_salience',
                     'lowlevel.spectral_centroid',
                     'lowlevel.spectral_rms',
                     'lowlevel.spectral_complexity',
                     'lowlevel.spectral_energy',
                     'lowlevel.spectral_kurtosis',
                     'lowlevel.spectral_entropy',
                     'lowlevel.spectral_rolloff',
                     'lowlevel.spectral_spread',
                     'lowlevel.spectral_skewness',
                     'lowlevel.spectral_entropy',
                     'lowlevel.spectral_strongpeak',
                     'lowlevel.zerocrossingrate',
                     'lowlevel.spectral_flux',
                     'lowlevel.spectral_spread',
                     'lowlevel.hfc',
                     'tonal.chords_strength',
                     'tonal.hpcp_crest',
                     'tonal.hpcp_entropy',
                     'lowlevel.loudness_ebu128.momentary',
                     'lowlevel.loudness_ebu128.short_term',
                    #  'lowlevel.silence_rate_20dB',
                    #  'lowlevel.silence_rate_30dB',
                     'lowlevel.silence_rate_60dB',
                     'lowlevel.spectral_energyband_low',
                     'lowlevel.spectral_energyband_middle_low',
                     'lowlevel.spectral_energyband_middle_high',
                     'lowlevel.spectral_energyband_high',
                     # Newly added
                    #  'rhythm.beats_loudness'
                     ]


In [16]:
print("dur:", relevant_episodes[0].duration)
for trait in relevant_features:
    print(trait.split('.')[1], relevant_episodes[0].features_frames[trait].shape)
    # trait_name = trait.split(["."])
    # 1024, 2048
print((relevant_episodes[0].features_frames['lowlevel.dissonance'].shape[0]-1) * 2048 * .5/44100)

685.8879818594104
(29540,)
dissonance
(29540,)
melbands_crest
(29540,)
melbands_flatness_db
(29540,)
pitch_salience
(29540,)
spectral_centroid
(29540,)
spectral_rms
(29540,)
spectral_complexity
(29540,)
spectral_energy
(29540,)
spectral_kurtosis
(29540,)
spectral_entropy
(29540,)
spectral_rolloff
(29540,)
spectral_spread
(29540,)
spectral_skewness
(29540,)
spectral_entropy
(29540,)
spectral_strongpeak
(29540,)
zerocrossingrate
(29540,)
spectral_flux
(29540,)
spectral_spread
(29540,)
hfc
(14771,)
chords_strength
(14771,)
hpcp_crest
(14771,)
hpcp_entropy
(6856,)
loudness_ebu128
(6830,)
loudness_ebu128
(29540,)
silence_rate_20dB
(29540,)
silence_rate_30dB
(29540,)
silence_rate_60dB
(29540,)
spectral_energyband_low
(29540,)
spectral_energyband_middle_low
(29540,)
spectral_energyband_middle_high
(29540,)
spectral_energyband_high
(1281,)
beats_loudness
685.8942403628118


In [27]:
print(relevant_episodes[0].duration)
for episode in relevant_episodes:
    dur = episode.duration
    features = episode.features_frames
    bpms_ests = episode.bpm_estimates
    bpms_local = episode.local_bpm
    print("Duration", dur)
    # print("Ests:", bpms_ests.shape)
    print("Preds:", 5*bpms_local.shape[0])
    print("Local:", bpms_local.shape)
    print("Est per frame:", dur / (5*bpms_ests.shape[0]))
    print("Local per frame:", dur / (5*bpms_local.shape[0]))
    print("")
    # print(loudness.shape)
    # print(dissonance.shape[0] / loudness.shape[0])

# for trait in relevant_features:
#     print(relevant_episodes[0].features_frames[trait].shape)
#     print(trait.split('.')[1])
#     # trait_name = trait.split(["."])
#     # 1024, 2048
# print((relevant_episodes[0].features_frames['lowlevel.dissonance'].shape[0]-1) * 2048 * .5/44100)

685.8879818594104
Duration 685.8879818594104
Preds: 685
Local: (137,)
Est per frame: 0.5812610015757715
Local per frame: 1.001296323882351

Duration 649.8568707482993
Preds: 650
Local: (130,)
Est per frame: 0.5261998953427525
Local per frame: 0.9997798011512298

Duration 650.0479818594105
Preds: 650
Local: (130,)
Est per frame: 0.42348402726997425
Local per frame: 1.0000738182452469

Duration 664.0319727891157
Preds: 665
Local: (133,)
Est per frame: 0.3794468415937804
Local per frame: 0.9985443199836326

Duration 650.9249886621316
Preds: 650
Local: (130,)
Est per frame: 0.6710566893424037
Local per frame: 1.0014230594802025

Duration 686.0159863945578
Preds: 685
Local: (137,)
Est per frame: 0.6860159863945577
Local per frame: 1.0014831918168727

Duration 674.5279818594104
Preds: 675
Local: (135,)
Est per frame: 0.41637529744408047
Local per frame: 0.9993007138657931

Duration 646.9759863945578
Preds: 645
Local: (129,)
Est per frame: 0.3242987400473974
Local per frame: 1.003063544797764

In [10]:
%%add_to Audio
def __create_metronome_file(self):
    out_dir = "audio_beats"
    if self.chopped:
        out_filename = f"{self.filename} {self.start},{self.end} processed{self.ext}"
    else:
        out_filename = self.filename + " processed" + self.ext
    self.metronome_file = os.path.join(out_dir, out_filename)
    if os.path.exists(self.metronome_file):
        # print(f"\tAlready created metronome file")
        return
    print(f"\tCreating metronome file at {self.metronome_file}")
    levelExtractor = es.LevelExtractor()
    marker = es.AudioOnsetsMarker(onsets=self.beats, type='beep')
    marked_audio = marker(self.es_loader)
    es.MonoWriter(filename=self.metronome_file)(marked_audio)
    # print("Audio with beat positions saved to", self.metronome_file)
def get_beat_diagram(self):
    # plot(self.es_loader)
    plt.rcParams['figure.figsize'] = (15, 6)  
    plot(self.es_loader)
    for beat in self.beats:
        plt.axvline(x=beat*44100, color='red')
        # plt.axvline(x=beat, color='red')
    plt.xlabel('Time (samples)')
    plt.title("Audio waveform and the estimated beat positions")
    show()
def get_bpm_histograms(self):
    peak1_bpm, peak1_weight, peak1_spread, peak2_bpm, peak2_weight, peak2_spread, histogram = \
        es.BpmHistogramDescriptors()(self.beats_intervals)

    print("Overall BPM (estimated before): %0.1f" % self.avg_bpm)
    print("First histogram peak: %0.1f bpm" % peak1_bpm)
    print("Second histogram peak: %0.1f bpm" % peak2_bpm)

    fig, ax = plt.subplots()
    ax.bar(range(len(histogram)), histogram, width=1)
    ax.set_xlabel('BPM')
    ax.set_ylabel('Frequency of occurrence')
    plt.title("BPM histogram")
    ax.set_xticks([20 * x + 0.5 for x in range(int(len(histogram) / 20))])
    ax.set_xticklabels([str(20 * x) for x in range(int(len(histogram) / 20))])
    plt.show()
def get_bpm_stats(self):
    print("BPM:", self.avg_bpm)
    print("Beat positions (sec.):", self.beats)
    print("Beat estimation confidence:", self.beats_confidence) 
def get_pred_diagram(self):
    classes = musicnn_metadata['classes']
    # plt.rcParams['figure.figsize'] = (15, 25)  
    # plt.rcParams['figure.autolayout'] = True
    # ax.matshow(... , aspect=1.5) #"auto"
    plt.figure(figsize=(15, 45))  
    plt.matshow(self.preds.T, aspect=10)
    plt.imshow(self.preds.T, aspect=10)
    plt.title('taggram')
    plt.yticks(np.arange(len(classes)), classes)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.show()
# Essentia graphs

In [ ]:
!curl -SLO https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb
!curl -SLO https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.json
musicnn_metadata = json.load(open('msd-musicnn-1.json', 'r'))
for k, v in musicnn_metadata.items():
    print('{}: {}'.format(k, v))

# !curl -SLO https://essentia.upf.edu/models/classifiers/genre_dortmund/genre_dortmund-musicnn-msd-2.pb
# !curl -SLO https://essentia.upf.edu/models/classifiers/genre_dortmund/genre_dortmund-musicnn-msd-2.json
# !curl -SLO https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/fsd-sinet-vgg42-tlpf_aps-1.pb
# !curl -SLO https://essentia.upf.edu/models/audio-event-recognition/fsd-sinet/fsd-sinet-vgg42-tlpf_aps-1.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3123k  100 3123k    0     0   663k      0  0:00:04  0:00:04 --:--:--  685k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3287  100  3287    0     0   5474      0 --:--:-- --:--:-- --:--:--  5469
name: MSD MusiCNN
type: auto-tagging
link: https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb
version: 1
description: prediction of the top-50 tags in the dataset
author: Pablo Alonso
email: pablo.alonso@upf.edu
release_date: 2020-03-31
framework: tensorflow
framework_version: 1.15.0
classes: ['rock', 'pop', 'alternative', 'indie', 'electronic', 'female vocalists', 'dance', '00s', 'alternative rock', 'jazz', 'beautiful', 'metal', 'chillout', 'male vocalists', 'classic rock', 'soul', 'indie rock', 'Mellow', 

Exporting Data

In [ ]:
frame_audio_data = defaultdict(lambda: defaultdict(float))

count=0

hop_prop = .5
frame_size = 2048/44100
for i, episode in enumerate(relevant_episodes):
    title = episode.title
    dur = episode.duration
    features = episode.features_frames
    frames = math.ceil(episode.duration / 5)
    
    cur_times = defaultdict(float)
    cur_indeces = defaultdict(int)
  
    print(title)
    print(dur)
    for frame in range(frames):
        # print(f"\tFrame {frame} started")
        frame_audio_data[count]["Category"] = title
        frame_audio_data[count]["Frame"] = frame

        for trait in relevant_features:
            # print(f"\t\tTrait {trait} started")

            skip_conversion = False
            length = features[trait].shape[0]
            # if "tonal" in trait:
            #     print(dur/2, length * 2048 * .5/44100)
            # print(trait, length/2)
            
            if (
                abs(dur - (
                    length * frame_size * hop_prop )
                ) <= 1
                ):
                    multiplier = 1
            elif (
                abs(dur/2 - (
                    length * frame_size * hop_prop )
                ) <= 1
                ):
                    multiplier = 2
            elif (
                abs(dur - length/10) <= 5
                ):
                divisor = 10
                skip_conversion = True
            elif (trait == "rhythm.beats_loudness"):
                pass
            else:
                print("ERROR")
                print(dur, length)
                print(trait)
                print(george)

            cur_trait_sum = 0
            cur_trait_count = 0
            while (cur_indeces[trait] < length 
                    and cur_times[trait] < (frame+1) * 5 ):

                new_idx = cur_indeces[trait] 

                if not skip_conversion:
                    new_time = hop_prop * frame_size * multiplier
                else:
                     new_time = 1 / divisor
                
                cur_trait_sum += features[trait][new_idx] 
                cur_trait_count += 1

                cur_times[trait] += new_time
                cur_indeces[trait] += 1

            trait_name = '_'.join(trait.split('.')[1:])
            if cur_trait_count > 0:
                frame_audio_data[count][trait_name] = (
                    cur_trait_sum / cur_trait_count
                )
            

            # print(f"\t\tTrait {trait} finished")
        
        # print(f"\tFrame {frame} finished")

        count += 1

    
    print(f"\tAll frames of {episode} finished")
    
    
    # for trait in relevant_features:
    #     length = features[trait].shape[0]
    #     if abs(length - cur_indeces[trait] > 2) or abs(dur - cur_times[trait] > 2):
    #         print(trait)
    #         print(length, cur_indeces[trait])
    #         print(dur, cur_times[trait])

    print(count)


Coping
685.8879818594104
	All frames of <__main__.Audio object at 0x7fe51c6f8460> finished
138
Death
650.0479818594105
	All frames of <__main__.Audio object at 0x7fe41832d4f0> finished
269
Nightmares
664.0319727891157
	All frames of <__main__.Audio object at 0x7fe41832daf0> finished
402
Bliss
650.9249886621316
	All frames of <__main__.Audio object at 0x7fe41832d8b0> finished
533
Dreams
686.0159863945578
	All frames of <__main__.Audio object at 0x7fe41832da30> finished
671
Family
674.5279818594104
	All frames of <__main__.Audio object at 0x7fe41832da00> finished
806
Conflict
646.9759863945578
	All frames of <__main__.Audio object at 0x7fe41832db50> finished
936
Paradise
675.0399773242631
	All frames of <__main__.Audio object at 0x7fe41832d1c0> finished
1072
Love
673.022947845805
	All frames of <__main__.Audio object at 0x7fe41832d280> finished
1207


In [10]:

max_features = max([episode.features_frames['lowlevel.dissonance'].shape[0] for episode in relevant_episodes])

# hop_prop = .5
# frame_size = 2048/44100
count = 0
for i, episode in enumerate(relevant_episodes):
    title = episode.title
    dur = episode.duration
    features = episode.features_frames
    frames = math.ceil(episode.duration / 5)
    
    cur_times = defaultdict(float)
    cur_indeces = defaultdict(int)
  

    temp = 1    
    
    # print(title)
    # print(dur)
    for frame in range(frames-1):
        # print(f"\tFrame {frame} started")
        frame_audio_data[count]["Category"] = title
        frame_audio_data[count]["Frame"] = frame
        frame_audio_data[count]["Estimated BPM"] = episode.local_bpm[frame]
        count += 1

    # print(count, frames, episode.local_bpm.shape[0])
    if frames <= episode.local_bpm.shape[0]:
        frame_audio_data[count]["Category"] = title
        frame_audio_data[count]["Frame"] = frame
        frame_audio_data[count]["Estimated BPM"] = episode.local_bpm[frames-1]
        count += 1
    else: 
        frame_audio_data[count]["Category"] = title
        frame_audio_data[count]["Frame"] = frame
        frame_audio_data[count]["Estimated BPM"] = episode.local_bpm[frames-2]
        count += 1


    # 
    # print(f"\tAll frames of {episode} finished")
    print(count)
    
    # for trait in relevant_features:
    #     length = features[trait].shape[0]
    #     if abs(length - cur_indeces[trait] > 2) or abs(dur - cur_times[trait] > 2):
    #         print(trait)
    #         print(length, cur_indeces[trait])
    #         print(dur, cur_times[trait])


138
269
402
533
671
806
936
1072
1207


In [22]:
overall_audio_data = defaultdict(lambda: defaultdict(float))

for episode in relevant_episodes:
    e = episode.title

    overall_audio_data[e]["Season"] = episode.season
    overall_audio_data[e]["Episode"] = episode.episode
    overall_audio_data[e]["Duration"] = episode.end - episode.start
    overall_audio_data[e]["Average BPM"] = episode.avg_bpm
    overall_audio_data[e]["Danceability"] = episode.danceability[0]

    for trait in relevant_overall_features:
        trait_name = '_'.join(trait.split('.')[1:])
        overall_audio_data[e][trait_name] = episode.features[trait]


    
    # beats_dict[e] = np.zeros(max_length)
    # beats = episode.beats
    # for i, beat in enumerate(beats):
    #     beats_dict[e][i] = beat

    # beat_lengths_dict[e] = np.zeros(max_length)
    # beat_lengths = episode.beats_intervals
    # for i, beat_length in enumerate(beat_lengths):
    #     beat_lengths_dict[e][i] = beat

In [1]:
overall_class_data = defaultdict(lambda: defaultdict(float))
c_labels = musicnn_metadata['classes']

for episode in relevant_episodes:
    e = episode.title

    overall_class_data[e]["Season"] = episode.season
    overall_class_data[e]["Episode"] = episode.episode
    try:
        preds = episode.preds
    except:
        episode.get_preds()
        preds = episode.preds
    
    for i, trait in enumerate(preds.T):
        label = c_labels[i]
        overall_class_data[e][label + ".Mean"] = np.mean(trait)
        overall_class_data[e][label + ".Stdev"] = np.std(trait)



NameError: name 'defaultdict' is not defined

In [ ]:
# print(frame_audio_data[0])

defaultdict(<class 'float'>, {'Category': 'Coping', 'Frame': 0, 'dissonance': 0.48822254397802883, 'melbands_crest': 11.474709446231524, 'melbands_flatness_db': 0.13533737640224258, 'pitch_salience': 0.6614851147212364, 'spectral_centroid': 3667.93588369864, 'spectral_rms': 0.0014637534233415113, 'spectral_complexity': 6.018518518518518, 'spectral_energy': 0.004099926756477424, 'spectral_kurtosis': 2.1125200495675758, 'spectral_entropy': 8.26944535087656, 'spectral_rolloff': 5129.388427734375, 'spectral_spread': 11580078.549768519, 'spectral_skewness': 0.9546412400808616, 'spectral_strongpeak': 0.2307637175965993, 'zerocrossingrate': 0.16375732421875, 'spectral_flux': 0.030406022621000863, 'hfc': 5.978727630437904, 'chords_strength': 0.4174872404999203, 'hpcp_crest': 6.831147352854411, 'hpcp_entropy': 2.311590164899826, 'loudness_ebu128_momentary': -67.41319308561437, 'loudness_ebu128_short_term': -29.757609423469095, 'silence_rate_60dB': 0.7916666666666666, 'spectral_energyband_low': 

In [13]:
frame_audio_df = pd.DataFrame(frame_audio_data).T

In [15]:
# frame_audio_df.shape
# print(frame_audio_df.iloc[2])
# print(frame_audio_df.iloc[135])
print(frame_audio_df.iloc[-1])

Category                                     Love
Frame                                         133
dissonance                               0.347299
melbands_crest                          19.334415
melbands_flatness_db                     0.537683
pitch_salience                           0.482507
spectral_centroid                      359.409131
spectral_rms                             0.004225
spectral_complexity                      5.206107
spectral_energy                          0.020715
spectral_kurtosis                       81.952796
spectral_entropy                          5.01604
spectral_rolloff                       401.898333
spectral_spread                    1672515.717557
spectral_skewness                        7.279857
spectral_strongpeak                      0.994632
zerocrossingrate                         0.016572
spectral_flux                            0.079262
hfc                                        5.5432
chords_strength                          0.498234


In [14]:
frame_audio_df_copy = frame_audio_df.copy()

In [175]:
max(frame_audio_df_copy["Estimated BPM"])

232.0

In [ ]:
# z['c'] = z.apply(lambda row: 0 if row['b'] in (0,1) else row['a'] / math.log(row['b']), axis=1)

In [16]:
frame_audio_df_copy.rename(columns={"spectral_energyband_high": "treble volume",
                    "spectral_energyband_middle_high": "high midrange volume",
                    "spectral_energyband_middle_low": "low midrange volume",
                    "spectral_energyband_low": "bass volume",
                    "spectral_rms": "average overall loudness",
                    "dissonance": "dissonance",
                    "pitch_salience": "proiminence of pitches",
                    "spectral_flux": "timbre",
                    "zerocrossingrate": "percussiveness",
                    "melbands_flatness_db": "cleanliness of timbre",
                    "spectral_centroid": "brightness (normalized)",
                    "hfc": "brightness (unnormalized)",
                    "spectral_complexity": "richness of timbre",
                    "loudness_ebu_128.momentary": "average instantaneous loudness",
                    "loudness_ebu_128.short_term": "average segment loudness",
                    "chords_strength": "strength of chords",
                    "silence_rate_60dB": "\% of frames silent",
                    "hpcp_crest": "pitch max to min ratio",
                    "hpcp_entropy": "pitch ambiguity",
                    "spectral_skewness": "disparity b/w highs and lows",
                    "spectral_spread": "noisiness (spectral spread)",
                    "spectral_rolloff": "frequency of 85th percentile",
                    "spectral_energy": "volume variation between frequency levels 1",
                    "spectral_entropy": "volume variation between frequency levels 2",
                    "spectral_strongpeak": "strongest frequency",
                    "Estimated BPM": "bpm",
                    "melbands_crest": "max to mean ratio of volume",
                   }
                   , inplace=True)



In [17]:
order = ["Category", 
         "Frame", 
                    "BPM",
                    "bass volume",
                    "low midrange volume",
                    "high midrange volume",
                    "treble volume",
                    "average overall loudness",
                    "dissonance",
                    "timbre",
                    "percussiveness",
                    "proiminence of pitches",
                    "cleanliness of timbre",
                    "brightness (normalized)",
                    "brightness (unnormalized)",
                    "richness of timbre",
                    "average instantaneous loudness",
                    "average segment loudness",
                    "strength of chords",
                    "\% of frames silent",
                    "pitch max to min ratio",
                    "pitch ambiguity",
                    "disparity b/w highs and lows",
                    "noisiness (spectral spread)",
                    "frequency of 85th percentile",
                    "volume variation between frequency levels 1",
                    "volume variation between frequency levels 2",
                    "strongest frequency",
                    "max to mean ratio of volume"
]
frame_audio_df_copy.reindex(columns=order)

,Category,Frame,BPM,bass volume,low midrange volume,high midrange volume,treble volume,average overall loudness,dissonance,timbre,...,\% of frames silent,pitch max to min ratio,pitch ambiguity,disparity b/w highs and lows,noisiness (spectral spread),frequency of 85th percentile,volume variation between frequency levels 1,volume variation between frequency levels 2,strongest frequency,max to mean ratio of volume
0,Coping,0,NaN,0.001735,0.00117,0.000891,0.000518,0.001464,0.488235,0.030406,...,0.791667,6.831147,2.31159,0.956618,11601885.239583,5145.239258,0.0041,8.269075,0.230764,11.47871
1,Coping,1,NaN,0.01913,0.006996,0.001789,0.000393,0.004067,0.480733,0.092655,...,0.023256,9.256066,2.761567,1.289966,4469414.411628,811.851835,0.027587,7.741605,0.348847,15.605809
2,Coping,2,NaN,0.022049,0.007433,0.001746,0.000514,0.004185,0.482908,0.100665,...,0.018605,9.63835,2.794344,1.189855,4481680.613953,1050.519849,0.03075,7.799084,0.314333,16.136812
3,Coping,3,NaN,0.022124,0.007177,0.002074,0.000667,0.004267,0.484225,0.100593,...,0.027778,9.048986,2.884601,1.152443,4470896.472801,1111.950684,0.030824,7.85034,0.286442,16.375185
4,Coping,4,NaN,0.024717,0.004034,0.001437,0.000192,0.003548,0.486454,0.090496,...,0.009302,9.869887,2.736047,1.219275,4161838.037209,1303.209484,0.029255,7.868261,0.245545,16.54086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,Love,130,NaN,0.018372,0.017634,0.00317,0.000362,0.005637,0.456472,0.091544,...,0.0,14.058182,1.90175,2.271838,3517337.284884,616.450536,0.03589,6.975602,0.704875,14.242252
1203,Love,131,NaN,0.013797,0.019678,0.003814,0.000441,0.005666,0.443929,0.085412,...,0.0,14.654656,1.826448,2.107563,4084031.878488,722.313772,0.035706,7.081142,0.838334,13.243608
1204,Love,132,NaN,0.009241,0.01122,0.000753,0.000035,0.003626,0.410826,0.048971,...,0.12963,14.269517,1.817307,3.900869,2136766.895255,401.753743,0.017321,6.001887,0.448999,15.65493
1205,Love,133,NaN,0.00256,0.001721,0.002831,0.000295,0.002249,0.443079,0.044243,...,0.572093,10.285934,2.376302,3.077009,2759045.31686,1315.929097,0.007288,6.748245,1.077118,15.10196


In [18]:
# frame_audio_df_copy = pd.DataFrame(frame_audio_data).T
frame_audio_df_copy.to_csv("audio_data/FINAL_frame_audio_layified.csv")

In [19]:
# frame_audio_df = pd.DataFrame(frame_audio_data).T
frame_audio_df.to_csv("audio_data/FINAL_frame_audio_unfiltered.csv")

In [23]:
overall_audio_df = pd.DataFrame(overall_audio_data).T
frame_audio_df_copy.to_csv("audio_data/FINAL_episode_audio.csv")

In [ ]:
overall_audio_df

In [ ]:
# trying to get localized tempo
sr = 11025


## new code stops here

## here

In [ ]:
c_labels = musicnn_metadata['classes']
class_dict = defaultdict(lambda: defaultdict(float))
class_means_dict = defaultdict(lambda: defaultdict(float))
traits_dict = defaultdict(lambda: defaultdict(float))
all_dict = defaultdict(lambda: defaultdict(float))
beats_dict = defaultdict(lambda: defaultdict(float))
beat_lengths_dict = defaultdict(lambda: defaultdict(float))
# dict mapping episodes to sub-dicts mapping traits to vector of values
# 50 values for each trait of a given song

max_length = max([len(episode.beats) for episode in all_audio])

for episode in all_audio:
    e = episode.title

    traits_dict[e]["Season"] = episode.season
    traits_dict[e]["Episode"] = episode.episode
    traits_dict[e]["Duration"] = episode.end - episode.start
    traits_dict[e]["Average BPM"] = episode.avg_bpm
    traits_dict[e]["Dynamic Complexity"] = episode.dynamic_complexity[0]
    traits_dict[e]["Danceability"] = episode.danceability[0]
    traits_dict[e]["Loudness"] = episode.dynamic_complexity[1]
    traits_dict[e]["Loudness Mean"] = np.mean(episode.levels)
    traits_dict[e]["Loudness Median"] = np.median(episode.levels)
    traits_dict[e]["Loudness Stdev"] = np.std(episode.levels)
    traits_dict[e]["Loudness Max"] = np.max(episode.levels)
    traits_dict[e]["Loudness Min"] = np.min(episode.levels)
    traits_dict[e]["Loudness High Quantile"] = np.quantile(episode.levels, .75)
    traits_dict[e]["Loudness Low Quantile"] = np.quantile(episode.levels, .25)

    try:
        preds = episode.preds
    except:
        episode.get_preds()
        preds = episode.preds


    class_dict[e]["Season"] = episode.season
    class_means_dict[e]["Season"] = episode.season
    class_dict[e]["Episode"] = episode.episode
    class_means_dict[e]["Episode"] = episode.episode
    class_dict[e]["Duration"] = episode.end - episode.start
    class_means_dict[e]["Duration"] = episode.end - episode.start

    for i, trait in enumerate(preds.T):
        label = c_labels[i]
        class_means_dict[e][label] = np.mean(trait)
        class_dict[e][label + " Mean"] = np.mean(trait)
        class_dict[e][label + " Median"] = np.median(trait)
        class_dict[e][label + " Stdev"] = np.std(trait)
        class_dict[e][label + " Max"] = np.max(trait)
        class_dict[e][label + " Min"] = np.min(trait)
        class_dict[e][label + " High Quantile"] = np.quantile(trait, .75)
        class_dict[e][label + " Low Quantile"] = np.quantile(trait, .25)

    beats_dict[e] = np.zeros(max_length)
    beats = episode.beats
    for i, beat in enumerate(beats): 
        beats_dict[e][i] = beat

    beat_lengths_dict[e] = np.zeros(max_length)
    beat_lengths = episode.beats_intervals
    for i, beat_length in enumerate(beat_lengths): 
        beat_lengths_dict[e][i] = beat

In [27]:
five_s_tag_df = pd.DataFrame(five_s_tags).T
five_s_tag_df.rename(index={"Dan Deacon When I Was Done Dying": "Dying"}, inplace=True)
five_s_tag_df.to_csv("audio_data/five_s_tags.csv")

In [ ]:
classification_df = pd.DataFrame(class_dict).T
classification_means_df = pd.DataFrame(class_means_dict).T
traits_df = pd.DataFrame(traits_dict).T
full_df = pd.concat([traits_df, classification_df], axis=1).drop_duplicates()
full_df = full_df.loc[:,~full_df.columns.duplicated()]
beats_df = pd.DataFrame(beats_dict).T
beat_lengths_df = pd.DataFrame(beat_lengths_dict).T

In [ ]:
full_df.rename(index={"Dan Deacon When I Was Done Dying": "Dying"}, inplace=True)
beats_df.rename(index={"Dan Deacon When I Was Done Dying": "Dying"}, inplace=True)
beat_lengths_df.rename(index={"Dan Deacon When I Was Done Dying": "Dying"}, inplace=True)
classification_df.rename(index={"Dan Deacon When I Was Done Dying": "Dying"}, inplace=True)
classification_means_df.rename(index={"Dan Deacon When I Was Done Dying": "Dying"}, inplace=True)

In [178]:
classification_df.to_csv("audio_data/classification.csv")
classification_means_df.to_csv("audio_data/classification[means_only].csv")
traits_df.to_csv("audio_data/traits.csv")
full_df.to_csv("audio_data/all_data.csv")
beats_df.to_csv("audio_data/beats.csv")
beat_lengths_df.to_csv("audio_data/beat_lengths.csv")

In [96]:
classification_means_df.columns

Index(['Season', 'Episode', 'Duration', 'rock', 'pop', 'alternative', 'indie',
       'electronic', 'female vocalists', 'dance', '00s', 'alternative rock',
       'jazz', 'beautiful', 'metal', 'chillout', 'male vocalists',
       'classic rock', 'soul', 'indie rock', 'Mellow', 'electronica', '80s',
       'folk', '90s', 'chill', 'instrumental', 'punk', 'oldies', 'blues',
       'hard rock', 'ambient', 'acoustic', 'experimental', 'female vocalist',
       'guitar', 'Hip-Hop', '70s', 'party', 'country', 'easy listening',
       'sexy', 'catchy', 'funk', 'electro', 'heavy metal', 'Progressive rock',
       '60s', 'rnb', 'indie pop', 'sad', 'House', 'happy'],
      dtype='object')

More
Here

In [154]:
# print(all_audio[0].es_loader.shape)
# dur1 = len(all_audio[0].es_loader)/44100

# test_loader = es.MonoLoader(filename=all_audio[0].file, sampleRate=16000)()
# dur2 = len(test_loader)/16000
# print(dur1, dur2)
# test_preds = es.TensorflowPredictMusiCNN( graphFilename='msd-musicnn-1.pb')(test_loader)

print(all_audio[0].beats.shape[0]/44100)
# print(test_preds.shape[0]/16000)
# print(all_audio[0].preds.shape[0]/16000)
print(all_audio[0].preds.shape[0])
print(all_audio[0].filename)

# print(es.Duration(all_audio[0].es_loader))
# max_length_beats = max([len(episode.beats)*16000 for episode in all_audio])
# max_length_class = max([len(episode.preds)*44100 for episode in all_audio])
# for episode in all_audio:
#     print(len(episode.beats)/44100, len(episode.preds)/16000)
#     print("\t",(len(episode.beats)/44100)/(len(episode.preds)/16000))
#     print("\t",(len(episode.beats)/44100)-(len(episode.preds)/16000))
# print(max_length_beats, max_length_class)

0.030249433106575963
459
Off the Air - S10E01 - Coping [WEBDL-1080p]


In [ ]:
# def top_from_average(data, top_n=10):
#     av = np.mean(data, axis=0)
#     sorting = np.argsort(av)[::-1]
#     return sorting[:top_n], [av[i] for i in sorting]

# ep.get_events()

# # Read the metadata
# metadata_file = "fsd-sinet-vgg42-tlpf_aps-1.json"
# metadata = json.load(open(metadata_file, "r"))
# labels = metadata["classes"]

# # Compute the top-n labels and predictions
# top_n, averages = top_from_average(predictions, top_n=15)
# top_labels = [labels[i] for i in top_n]
# top_labels_with_av = [
#     f"{label} ({av:.3f})" for label, av in zip(top_labels, averages)
# ]

# top_predictions = np.array([predictions[i, :] for i in top_n])

# # Generate plots and improve formatting
# matfig = plt.figure(figsize=(8, 3))
# plt.matshow(top_predictions, fignum=matfig.number, aspect="auto")

# plt.yticks(np.arange(len(top_labels_with_av)), top_labels_with_av)
# locs, _ = plt.xticks()
# ticks = np.array(locs // 2).astype("int")
# plt.xticks(locs[1: -1], ticks[1: -1])
# plt.tick_params(
#     bottom=True, top=False, labelbottom=True, labeltop=False
# )
# plt.xlabel("(s)")

# plt.savefig("activations.png", bbox_inches='tight')
# ep.get_events()


In [ ]:
ep0 = Audio(AUDIO, audio_files[0]) 
ep1 = Audio(AUDIO, audio_files[1]) 
ep2 = Audio(AUDIO, audio_files[2]) 

# new_preds0 = es.TensorflowPredictMusiCNN(patchHopSize=0, 
#             graphFilename='msd-musicnn-1.pb')(ep0.pred_loader)
# print(new_preds0.shape[0], ep0.preds.shape[0], ep0.duration)

# new_preds1 = es.TensorflowPredictMusiCNN(patchHopSize=0,
#             graphFilename='msd-musicnn-1.pb')(ep1.pred_loader)
# print(new_preds1.shape[0], ep1.preds.shape[0], ep1.duration)

# new_preds2 = es.TensorflowPredictMusiCNN(patchHopSize=0,
#             graphFilename='msd-musicnn-1.pb')(ep2.pred_loader)
# print(new_preds2.shape[0], ep2.preds.shape[0], ep2.duration)

In [47]:
new_preds0 = es.TensorflowPredictMusiCNN(patchHopSize=155,patchSize=311,
            graphFilename='msd-musicnn-1.pb')(ep0.pred_loader)
print(new_preds0.shape[0], ep0.preds.shape[0], ep0.duration)

new_preds1 = es.TensorflowPredictMusiCNN(patchHopSize=155,patchSize=311,
            graphFilename='msd-musicnn-1.pb')(ep1.pred_loader)
print(new_preds1.shape[0], ep1.preds.shape[0], ep1.duration)

new_preds2 = es.TensorflowPredictMusiCNN(patchHopSize=155,patchSize=311,
            graphFilename='msd-musicnn-1.pb')(ep2.pred_loader)
print(new_preds2.shape[0], ep2.preds.shape[0], ep2.duration)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `msd-musicnn-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `msd-musicnn-1.pb`


275 459 685.8879818594104
288 481 717.599977324263
265 443 660.9909297052154


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `msd-musicnn-1.pb`


In [ ]:
# for i in range(audio.es_loader.shape[0]/44100 - (audio.preds.shape[0]*1.5-4.5)):


In [39]:
with open('genre_dortmund-musicnn-msd-2.json', 'r') as json_file:
    metadata = json.load(json_file)
graph_file_name = 'genre_dortmund-musicnn-msd-2.pb'

for i in range(5):
    activations = es.TensorflowPredictMusiCNN(graphFilename=graph_file_name)(all_audio[i].pred_loader)
    for label, probability in sorted(zip(metadata['classes'], activations.mean(axis=0)), key = lambda t:t[1], reverse=True)[:3]:
        print(f'{label}: {100 * probability:.1f}%')
    print("")

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_dortmund-musicnn-msd-2.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_dortmund-musicnn-msd-2.pb`


electronic: 49.5%
jazz: 16.4%
raphiphop: 13.4%

electronic: 55.1%
jazz: 21.3%
raphiphop: 8.0%



[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_dortmund-musicnn-msd-2.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_dortmund-musicnn-msd-2.pb`


electronic: 56.3%
jazz: 15.2%
raphiphop: 12.8%

electronic: 49.4%
jazz: 18.6%
raphiphop: 17.6%

electronic: 43.4%
rock: 22.4%
jazz: 18.9%



[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_dortmund-musicnn-msd-2.pb`
